In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
# Chargement du dataset Iris depuis une URL
url = "https://raw.githubusercontent.com/jundewu10/Iris-dataset/main/iris.csv"

# Télécharger le CSV avec pandas
df = pd.read_csv(url)
print(df.head())

# Télécharger le contenu brut du CSV
import requests
response = requests.get(url)
raw_data = response.content 

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<p>Connaitre l'encodage du fichier source .csv</p>

</div>

In [ ]:
import chardet
with open("source.csv", rb) as fichier_brut:
    encodage = chardet.detect(fichier_brut.read(1000))
print(encodage)
# # >>> { 'encoding' : 'UTF-8-SIG', 'confidence': 1.0, 'language' : ''}

df = pd.read_csv("source.csv", encoding='utf-sig-8')


<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<p>Connaitre l'encodage du fichier source binaire</p>

</div>

In [ ]:
import chardet
# Détecter l'encodage d'un binaire
encodage = chardet.detect(raw_data)
print(encodage)

<div style="
    background-color: #c8a043ff; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<h3>Valeurs manquantes</h3>
</div>

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<p>Récupération des noms de variables vides</p>
</div>

In [ ]:
empty_variables = df.columns[df.isna().all()].tolist()
empty_variables

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<p>Remplacement des manquants par zéro, voir librairie Missingno</p>
</div>

In [ ]:
# Option 1 : réassigner directement la colonne
df['sepal_length'] = df['sepal_length'].fillna(0)

# Option 2 : utiliser fillna sur le DataFrame entier
df.fillna({'sepal_length': 0}, inplace=True)



<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<p>Taux de remplissage par variables</p>
</div>

In [ ]:
fill_rate = df.count() / len(df) * 100

<div style="
    background-color: #c8a043ff; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<h3>Statistiques descriptives</h3>
</div>


- `s` est une `pandas.Series`
- `df` est un `pandas.DataFrame`
- `x`, `y` sont des `numpy.ndarray`

| Indicateur               | pandas               | numpy                              |
| ------------------------ | -------------------- | ---------------------------------- |
| Moyenne                  | `s.mean()`           | `np.mean(x)`                       |
| Médiane                  | `s.median()`         | `np.median(x)`                     |
| Mode                     | `s.mode()`           | — *(pas natif)*                    |
| Minimum                  | `s.min()`            | `np.min(x)`                        |
| Maximum                  | `s.max()`            | `np.max(x)`                        |
| Étendue (max − min)      | `s.max() - s.min()`  | `np.ptp(x)`                        |
| Somme                    | `s.sum()`            | `np.sum(x)`                        |
| Variance                 | `s.var()`            | `np.var(x)`                        |
| Écart-type               | `s.std()`            | `np.std(x)`                        |
| Quantile                 | `s.quantile(q)`      | `np.quantile(x, q)`                |
| Percentile               | `s.quantile(q)`      | `np.percentile(x, q)`              |
| Coefficient de variation | `s.std() / s.mean()` | `np.std(x) / np.mean(x)`           |
| Asymétrie (skewness)     | `s.skew()`           | — *(via `scipy.stats.skew`)*       |
| Aplatissement (kurtosis) | `s.kurt()`           | — *(via `scipy.stats.kurtosis`)*   |
|Rang  |`df["Column"].rank()`  |`np.argsort(array)`|
| Covariance               | `s.cov(t)`           | `np.cov(x, y)`                     |
| Matrice de covariance    | `df.cov()`           | `np.cov(X, rowvar=False)`          |
| Corrélation (Pearson)    | `s.corr(t)`          | `np.corrcoef(x, y)`                |
| Matrice de corrélation   | `df.corr()`          | `np.corrcoef(X, rowvar=False)`     |
| Comptage (effectif)      | `s.count()`          | `x.size`                           |
| Valeurs uniques          | `s.nunique()`        | `np.unique(x).size`                |
| Fréquences               | `s.value_counts()`   | `np.unique(x, return_counts=True)` |


<div style="
    background-color: #c8a043ff; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<h3>Variables catégorielles</h3>
</div>

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<p>Récupération des variables n'ayant qu'une seule modalité</p>
</div>

In [ ]:
# dropna=False force à considérer les valeurs manquantes comme une modalité
unique_variables = df.columns[ (df.nunique(dropna=False) == 1 ).values].tolist()

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
<p>Regroupe les catégories rares</p>
</div>

In [ ]:
def group_rare_categorie(df, column, threshold=5, new_label='Other'):
    """
    Pour un DataFrame (df)
    la fonction regroupe les catégories rares
    fixées par un seuil (threshold) d'une variable catégorielle (column)
    sous l'étiquette 'labenew_l'
    """
    # calcul de la fréquence des modalités
    frequency = df[column].value_counts()
    # identification des catégories rares
    rare_categories = frequency[frequency < threshold].index
    # remplacement des catégories rares par 'Other'
    df[columns] = df[columns].replace(rare_categories, new_label) 

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h4>Tableau de contingence ou tableau croisé</h4>
<p>Explore les associations entres 2 variables</p>
</div>

In [ ]:
contengency_table = pd.crosstab(df['column_1'], df['column_2'])

<div style="
    background-color: #c8a043ff; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Indice de diversité</h3>
    <p>quantifient la répartition et l'équilibre des catégories au sein dune variable ou d'un couple de variables</p>
</div>

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h4>Indice de diversité de Berger-Parker</h4>
    Proportion de la catégorie la plus importante<br>
    Plus il est proche de 0, plus il y a de diversité<br>
    Plus il est proche de 1, moins il y a de diversité</p>
</div>

In [ ]:
def berger_parker_index(df):
    """
    Calcule l'indice de Berger-Parker pour la variable catégorielle du tableau de contingence (df)
    Proportion de la catégorie la plus importante
    Plus il est proche de 1, plus la diversité est faible et inversement
    """
    # Normalisation par colonne
    proportion = df.div(df.sum(axis=1), axis=0)
    # Valeur maximale par ligne
    berger_index = proportion.max(axis=1)
    # affichage sous )-àforme de DataFrame
    return pd.DataFrame(
        berger_index, 
        columns=['Berger-Parker Index']
    )

# Calcul de l'indice de Berger-Parker
berger_parker_index = berger_parker_index(contengency_table)    

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h4>Indice de diversité de Simpson</h4>
    <p>Probabilité que 2 individus sélectionnés au hasard dans une population appartiennent à la même catégorie<br>
    Plus il est proche de 0, plus il y a de diversité<br>
    Plus il est proche de 1, moins il y a de diversité</p>
</div>

In [ ]:
def simpson_index(df):
    """
    Calcule l'indice de Simpson pour la variable catégorielle du tableau de contingence (df)
    Probabilité que deux individus pris au hasard appartiennent à la même catégorie
    Plus il est proche de 1, plus la diversité est faible et inversement
    """
    # Normalisation par colonne
    proportion = df.div(df.sum(axis=1), axis=0)
    # Somme des carrés des proportions par ligne
    simpson_index = (proportion ** 2).sum(axis=1)
    # affichage sous forme de DataFrame
    return pd.DataFrame(
        simpson_index, 
        columns=['Simpson Index']
    )
# Calcul de l'indice de Simpson
simpson_index = simpson_index(contengency_table)

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h4>Indice de diversité de Shannon-Weaver</h4>
    <p>Chaque proportion est multipuié par son logarithme<br>
    Donne plus de poids au catégories moins communes<br>
    vaut 0 s'il n'y a qu'une catégorie<br>
    plus il y a de catégories, plus cette indice peut être élevé</p>
</div>

In [ ]:
def shannon_index(df):
    """
    Calcule l'indice de Shannon pour la variable catégorielle du tableau de contingence (df)
    Mesure de l'incertitude associée au choix d'une catégorie au hasard
    Plus il est élevé, plus la diversité est grande
    """
    # Normalisation par colonne
    proportion = df.div(df.sum(axis=1), axis=0)
    # Calcul de l'indice de Shannon
    shannon_index = - (proportion * np.log(proportion + 1e-9)).sum(axis=1)
    # affichage sous forme de DataFrame
    return pd.DataFrame(
        shannon_index, 
        columns=['Shannon Index']
    )
# Calcul de l'indice de Shannon
shannon_index = shannon_index(contengency_table)

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Fonctions</h3>
<p>détection des outliers</p>
<ul><li>méthode des quartiles (écart interquartiles)</li><li>méthode des z-score</li>
</ul>
</div>

In [ ]:
def detect_outliers_iqr(data, factor=1.5):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    return (data < lower_bound) | (data > upper_bound)


def detect_outliers_z_score(data, threshold=3):
    z_scores = (data - data.mean()) / data.std()
    return np.abs(z_scores) > threshold

In [ ]:
# Création de données de test
np.random.seed(0)
data = np.random.normal(loc=0, scale=1, size=100)

# Ajout d'outliers
data[0] = 10
data[1] = -10

# Détection des outliers avec la méthode des quartiles
outliers_iqr = detect_outliers_iqr(pd.Series(data))
print("Outliers détectés par la méthode des quartiles:")
print(pd.Series(data)[outliers_iqr])

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Local Outlier Factor</h3>

</div>

In [ ]:
np.random.seed(0)

# matrice de données avec 100 échantillons et 2 variables
# random.randn génère des échantillons à partir d'une distribution normale entre 0 et 1
X = 0.6 * np.random.randn(100, 2)

# Insertion de quelques outliers
# random.uniform génère des valeurs aléatoires uniformément distribuées entre low et high
X_outliers = np.random.uniform(low=-5, high=5, size=(10, 2))

# Compilation verticale des données normales et des outliers
X = np.vstack([X, X_outliers])


In [ ]:

# Entrainement du modèle LOF
clf = LocalOutlierFactor(n_neighbors=10)

# Entrainement et prédiction
# Définin chaque point comme normal ou pas
# -1 pour les outliers et 1 pour les points normaux
y_pred = clf.fit_predict(X)

# Création d'un masque pour les outliers
outliers_mask = y_pred == -1

# outliers en bleu
plt.scatter(X[outliers_mask, 0], X[outliers_mask, 1], color='b', label='Outliers')
# points normaux en rouge
plt.scatter(X[~outliers_mask, 0], X[~outliers_mask, 1], color='r', label='Normal points')
plt.legend()
plt.show()

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Changement de la distribution</h3>

</div>

In [ ]:
# Transformation logarithmique
df['variable_log'] = np.log(['Variable'])

# Transformation de la racine carrée
df['variable_sqrt'] = np.sqrt(df['Variable'])

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Imputation Pandas</h3>
<ul>
<li>modale</li>
<li>médiane</li>
<li>moyenne</li>
</ul>
<p>Selon le taux de valeurs manquante,<br>l'imputation par la moyenne augmente le nombre d'outliers</p>
</div>

In [ ]:
# Remplir les valeurs manquantes avec la modalité la plus fréquente
# précise l'indicece [0] au cas ou il y a plusieurs modalités
df['variable'] = df['Variable'].fillna(df['Variable'].mode()[0])

# Imputation par la moyenne avec pandas
df['variable'] = df['Variable'].fillna(df['Variable'].mean())

# Imputation par la m&diane avec pandas
df['variable'] = df['Variable'].fillna(df['Variable'].median())


<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Imputation Scikit-learn</h3>
<ul>
<li>modale</li>
<li>médiane</li>
<li>moyenne</li>
</ul>
</div>

In [ ]:

from sklearn.impute import SimpleImputer

imputer_mode = SimpleImputer(strategy='most_frequent')
df['variable'] = imputer_mode.fit_transform(df[['Variable']])

imputer_mean = SimpleImputer(strategy='mean')
df['variable'] = imputer_mean.fit_transform(df[['Variable']])

imputer_median = SimpleImputer(strategy='median')
df['variable'] = imputer_median.fit_transform(df[['Variable']])

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Imputation par regression</h3>
<p>
Pour une variable cible Y contenant des valeurs manquantes :

Séparer les lignes avec Y connu et Y manquant.

Entraîner un modèle de régression Y ~ X sur les lignes complètes.

Prédire les valeurs manquantes à partir des autres variables.

Remplacer les NaN par ces prédictions.

C’est une imputation plus intelligente que mean/median/mode.
</p>
</div>

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10)
df_imputed = imputer.fit_transform(df)
df_imputed = pd.DataFrame(df_imputed, columns=df.columns)


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Exemple : variables numériques et catégorielles
numerical_cols = ['age', 'income', 'Variable']     # Variable à imputer
categorical_cols = ['genre', 'ville']

# Préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ("num", IterativeImputer(estimator=RandomForestRegressor(), max_iter=10), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

# Pipeline complet
pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", RandomForestRegressor())
])

# Entraînement du pipeline
pipeline.fit(df[numerical_cols + categorical_cols], df['Variable'])


In [ ]:
from sklearn.impute import KNNImputer

cols_with_missing = ['Variable']
imputer = KNNImputer(n_neighbors=5)
df[cols_with_missing] = imputer.fit_transform(df[cols_with_missing])

<div style="
    background-color: #439cc8; 
    color: #fff; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Bibliothèque d'Imputation</h3>
<p>
Fancyimpute

impyute
</p>
</div>